In [1]:
import sys
#sys.path.append('/Users/ManuelSerraNunes/Desktop/Tecnico_5_1S/IA/Artificial_Intelligence_IST/mini_project1')
sys.path.append('/Users/simaomoraessarmento/Dropbox/IST/S1/IA/Artificial_Intelligence_IST/mini_project1')
from go import *

In [2]:
#file_path = "Artificial_Intelligence_IST/mini_project1/example2.txt"
file_path = "/Users/simaomoraessarmento/Dropbox/IST/S1/IA/Artificial_Intelligence_IST/mini_project1/example2.txt"
file = open(file_path)

In [3]:
"""Loads a board from an opened file and returns the corresponding state"""

# 1. First line should contain:
#   - size of board
#   - next player to move
line1 = file.readline()
size, player = map(int, line1.split(' '))
print('Board size: {}x{}'.format(size, size))
print('Next player to move: ', player)

# 2. Load board into matrix
board = []
for row in range(size):
    board_row = file.readline().split('\n')[0]
    board.append([int(point) for point in board_row])
print(board)
    

Board size: 5x5
Next player to move:  1
[[0, 0, 0, 0, 1], [1, 0, 1, 0, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [0, 0, 0, 0, 0]]


In [7]:
def update_liberties(row, col, board, group_board, player, groups):
    """
    This function calculates the liberties for the given point,
    and updates the corresponding group information regarding so.
    
    :param row: row point position
    :param col: col point position
    :param board: matrix with board pieces
    :param group_board: matrix with board groups
    :param player: next player to move
    :param groups: dictionary with groups' information
    """
    group = group_board[row][col]
    
    if row - 1 >= 0:
        if board[row-1][col] == 0:
            groups[player][group].add_liberty((row-1, col))
    if row + 1 < len(board):     
        if board[row+1][col] == 0:
            groups[player][group].add_liberty((row+1, col))
    if col - 1 >= 0:
        if board[row][col-1] == 0:
            groups[player][group].add_liberty((row, col-1))
    if col + 1 < len(board):
        if board[row][col+1] == 0:
            groups[player][group].add_liberty((row, col+1))
    
            
def merge_groups(row, col, player, groups, left_neighbor_group, group_board):
    """
    This function is responsible for merging two connected groups initially
    identified as separated components.
    
    :param row: row point position
    :param col: col point position
    :param player: next player to move
    :param groups: dictionary with groups' information
    :param left_neighbor_group: group to merge
    :param group_board: matrix with board groups
    
    :return board elements to update on the board
    """
    group = group_board[row][col]
    
    # 1) merge elements
    elements_to_merge = groups[player][left_neighbor_group].elements
    groups[player][group].add_element(elements_to_merge)
    
    # 2) merge liberties (liberties count is updated internally)
    liberties_to_merge = groups[player][left_neighbor_group].liberties
    for liberty in liberties_to_merge:
        groups[player][group].add_liberty(liberty)
    
    return elements_to_merge

group_board = [[0] * len(board) for i in range(len(board))]
groups = {1:dict(), 2:dict()}
counters = {1:1, 2:2}


for row in range(len(board)):
    for col in range(len(board)):
        print('\n', row, col)
        player = board[row][col]
        
        # Check if current point is occupied. If not, it just skips to next point
        if board[row][col] != 0:
            """
            Component identification goes as follows:
            1) Look for a group neighbor on the row above. If existent, join group.
            2) Look for a group neighbor on the col on the left. If existent:
                2.1) If neighbor was found both on top and left, then merge left group
                     w/ top group
                2.2) If only left neighbor was found, then join current point to left
                     group.
            3) If no neighbor was found (same player piece in any of the neighboring points)
               then a new group is created
            """
                        
            if row - 1 >= 0:
                top_neighbor = board[row-1][col] 
                top_neighbor_group = group_board[row-1][col] 
    
                if top_neighbor_group != 0 and top_neighbor == player:
                    # updates group board
                    group_board[row][col] = top_neighbor_group
                    # adds element to group structure
                    new_elements = [(row, col)]
                    groups[player][group_board[row][col]].add_element(new_elements)
    
            if col - 1 >= 0:
                left_neighbor = board[row][col-1]
                left_neighbor_group = group_board[row][col-1]
                
                if left_neighbor_group != 0 and left_neighbor == player:
                    # verifies if groups must be merged (condition 2.1)
                    # by checking group_board[row][col] != 0, we ensure a group has been 
                    # attributed before
                    if group_board[row][col] != 0 and left_neighbor_group != group_board[row][col]:
                        # updates left neighbor's group elements
                        elements_to_merge= merge_groups(row, col, player, groups, left_neighbor_group, group_board)
                        # removes old group from dictionary
                        groups[player].pop(left_neighbor_group)
                        # updates group board
                        for old_element_row, old_element_col in elements_to_merge:
                            group_board[old_element_row][old_element_col] = group_board[row][col]
                    
                    # no neighbor on top, just join left neighbor's group (condition 2.2)
                    elif group_board[row][col] == 0:
                        # updates group board
                        group_board[row][col] = left_neighbor_group
                        # adds element to group structure
                        new_elements = [(row, col)]
                        groups[player][group_board[row][col]].add_element(new_elements)
            
            # if no merging neighbors were found, than just creates a new group
            if group_board[row][col] == 0:
                group_board[row][col] = counters[player]
                groups[player][counters[player]] = Group((row, col))
                counters[player] += 2
                
            # Finally, add information about group's liberties to the structure
            update_liberties(row, col, board, group_board, player, groups)
        print(group_board)


 0 0
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 0 1
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 0 2
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 0 3
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 0 4
[[0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 1 0
[[0, 0, 0, 0, 1], [3, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 1 1
[[0, 0, 0, 0, 1], [3, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 1 2
[[0, 0, 0, 0, 1], [3, 0, 5, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 1 3
[[0, 0, 0, 0, 1], [3, 0, 5, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 1 4
[[0, 0, 0, 0, 1], [3, 0, 5, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

 2 0
[[0, 0, 0, 0, 1], [3, 0, 5, 0, 1], [3, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0

In [8]:
print(groups[1][1].n_liberties)

10


In [22]:
groups[2][2].elements

[(1, 2), (1, 3)]